# Assessing Li-ion Cylindrical Cell Canister from XCT Slices

This notebook demonstrates a reproducible workflow for assessing the **can (canister)** of a Li-ion cylindrical cell from X-ray CT (XCT) slices. The can is a critical safety component: dents or out-of-roundness can indicate underlying mechanical or manufacturing issues.

**We extract:**
- **Outer/Inner Equivalent Diameters** (area-equivalent circle diameters)
- **Wall Thickness** statistics via distance transform (mean / min / max / σ)
- **Eccentricity** from an ellipse fit to the outer boundary
- **Denting (max inward deviation)** relative to the fitted ellipse

**Notebook map:**
1. Load an XCT slice and create a binary mask of the can wall
2. Extract inner/outer contours and visualize
3. Compute equivalent diameters (outer & inner)
4. Estimate wall thickness from inner wall to outer border (EDT)
5. Fit an ellipse to the outer contour (RANSAC) and compute eccentricity
6. Build an angular dent profile and report max dent
7. Visual summary and a synthetic data example with obvious defects

## 0) Setup & imports
Add the project root to `sys.path` (so we can import utilities), then import the scientific Python stack.

In [ ]:
import os, sys
CWD = os.getcwd()
if os.path.basename(CWD) == "notebooks":
    PROJECT_ROOT = os.path.abspath(os.path.join(CWD, ".."))
else:
    PROJECT_ROOT = CWD  # fallback if already at root
os.chdir(PROJECT_ROOT)
sys.path.append(os.path.join(PROJECT_ROOT, "utils"))
print(f"Correct Working Directory: {str(os.path.basename(os.getcwd()))=='battery_xct_workflows'}")

In [ ]:
# If you're running this notebook in Google Colab, install requirements:
import sys
if "google.colab" in sys.modules:
    !pip install -q -r https://raw.githubusercontent.com/MPJ-Imaging/battery_xct_workflows/HEAD/requirements.txt

# If you're running in a local Jupyter Notebook/Lab and see missing packages,
# you should uncomment and run the line below:
# !pip install -r requirements.txt

In [ ]:
import tifffile as tiff
import numpy as np
import skimage
from skimage.measure import find_contours, EllipseModel, ransac
from skimage.draw import polygon2mask
from scipy.ndimage import distance_transform_edt as dist_trans
from scipy.ndimage import convolve
import matplotlib.pyplot as plt
import math
import utils.plotting_utils as plot
import utils.can as cn

## 1) Load image & build a can mask
Load a single XCT slice and create a binary mask of the can wall. The threshold here is a simple global value—adjust for your data (or replace with your preferred segmentation method). Set `px` to your pixel size so printed values are in millimetres.

In [ ]:
# Read image and calcualte a mask
im = tiff.imread('data/cell_vol/im075.tif')
px = 0.03397 # mm
mask = im > 100

# Side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 10))
axes = axes.ravel()

# Iamge Mask Pair: image + overlay
axes[0].imshow(im, cmap="gray", interpolation="nearest")
axes[0].set_title("XCT Slice")
axes[0].axis("off")
plot.show_overlay(im, mask, axes[1], "XCT Slice + Can Mask")

plt.tight_layout()
plt.show()

## 2) Extract inner/outer contours (and masks)
We find subpixel contours using marching squares (`find_contours` at level 0.5), then identify **outer** and **inner** loops by area (shoelace formula). We then fill each contour into a mask with `polygon2mask`, as illustrated in the plot below.

In [ ]:
def extract_inner_outer_contours(mask, as_mask = False):
    contours = find_contours((mask).astype(float), 0.5)
    outer = max(contours, key=lambda c: 0.5*np.abs(np.sum(c[:,1]*np.roll(c[:,0],-1) - c[:,0]*np.roll(c[:,1],-1))))
    inner = min(contours, key=lambda c: 0.5*np.abs(np.sum(c[:,1]*np.roll(c[:,0],-1) - c[:,0]*np.roll(c[:,1],-1))))
    if as_mask == True:
        outer = polygon2mask(mask.shape, outer).astype(np.uint8)
        inner = polygon2mask(mask.shape, inner).astype(np.uint8)
    return outer, inner

outer, inner = extract_inner_outer_contours(mask, as_mask = True)
inner = inner * ~mask # Remove any 'wall' pixels from inner mask

fig, axes = plt.subplots(1, 1, figsize=(5, 5))
axes.imshow(outer+inner, interpolation="none")
axes.set_title("Inner + Outer Mask")
axes.axis("off")

plt.tight_layout()
plt.show()

## 3) Equivalent circle diameters (outer & inner)
Compute the **area-equivalent** diameters: \( D_{eq} = 2\sqrt{A/\pi} \). This is robust for large, nearly circular shapes and gives a single scalar per region.

In [ ]:
def equivalent_diameter_area(mask):
    area = np.sum(mask)
    return math.sqrt(area/math.pi)*2

OD = equivalent_diameter_area(outer)
print(f'Outer Diameter: {(OD*px):.2f} mm')

ID = equivalent_diameter_area(inner)
print(f'Inner Diameter: {(ID*px):.2f} mm')

## 4) Wall thickness from distance transform (EDT)
Extract one-pixel **outer border** and **inner wall** edges using a fast convolution trick. Then compute an EDT to the outer border and sample it at inner-wall pixels. Report mean / min / max / 3σ thickness (converted to mm for display).

In [ ]:
def surface_pixels_convolve(img):
    img8 = img.astype(np.uint8)
    s = convolve(img8, np.ones((3,3), np.uint8), mode='constant', cval=0)
    # interior: all 9 in the 3x3 (8-connected)
    interior = (img & (s == 9))
    return img & ~interior

outer_border = surface_pixels_convolve(outer)
outer_border = (outer_border == 1).astype(bool)

inner_wall = surface_pixels_convolve(inner)
inner_wall = (inner_wall == 1).astype(bool)

edt = dist_trans(np.where(outer_border == True, False, True))
thickness_vals = edt[inner_wall]
thick_mean, thick_min, thick_max, thick_std = thickness_vals.mean(), thickness_vals.min(), thickness_vals.max(), thickness_vals.std()

print(f'Mean Wall Thickness: {(thick_mean*px):.3f} mm')
print(f'Wall Thickness 3\u03C3: {(thick_std*3*px):.3f} mm')

## 5) Ellipse fitting (RANSAC) to the outer contour
Fit an ellipse to the **outer** boundary points using RANSAC for robustness to local dents/outliers. The model yields center `(xc, yc)`, semi-axes `(a, b)`, and rotation `phi` (radians).

In [ ]:
contours = find_contours((mask).astype(float), 0.5)
outer = max(contours, key=lambda c: 0.5*np.abs(np.sum(c[:,1]*np.roll(c[:,0],-1) - c[:,0]*np.roll(c[:,1],-1))))
pts_xy = np.column_stack([outer[:,1], outer[:,0]])  # (x,y)

model, inliers = ransac(
    pts_xy,
    EllipseModel,
    min_samples=5,
    residual_threshold=1.0,     # tunable
    max_trials=2000,
    # stop_probability=0.999,   # optional
)
xc, yc, a, b, phi = model.params

## 6) Eccentricity of the fitted ellipse
Compute eccentricity \( e = \sqrt{1-(b/a)^2} \) ensuring `a ≥ b`. This summarizes deviation from circularity (0 = circle; →1 = elongated).

In [ ]:
A, B = (a, b) if a >= b else (b, a)   # ensure A = major, B = minor
eccentricity = np.sqrt(max(0.0, 1.0 - (B/A)**2))  # guard tiny numerical negatives

print(f"Eccentricity : {eccentricity:.4f}")

## 7) Build a dent profile against the ellipse
For each boundary point, compute:
1. Its angle around the ellipse center, and **observed radius** \( r_{obs} \)
2. The ideal **ellipse radius** in that direction, \( r_{ell}(\theta) = \frac{ab}{\sqrt{(b\cos(\theta-\phi))^2 + (a\sin(\theta-\phi))^2}} \)
3. The dent depth \( \delta = r_{ell} - r_{obs} \) (positive = inward dent)
Then average \(\delta\) in angular bins for a clean profile.

In [ ]:
# radii & angles of observed contour w.r.t. the ellipse center
vx = pts_xy[:, 0] - xc
vy = pts_xy[:, 1] - yc
theta = np.arctan2(vy, vx)                     # [-pi, pi)
r_obs = np.hypot(vx, vy)

In [ ]:
# rotate the direction into the ellipse's principal frame
ct = np.cos(theta - phi)
st = np.sin(theta - phi)
r_elli = (a * b) / np.sqrt((b * ct) ** 2 + (a * st) ** 2)


In [ ]:
# dent depth (positive = inward dent; negative = outward bulge)
delta = r_elli - r_obs

### Bin the dent depths by angle and report the maximum
Use uniform angular bins, take the mean per bin (robust alternatives: median, or smoothing), and report the **maximum inward dent** in millimetres.

In [ ]:
# bin by angle for a uniform profile
n_bins = 180
bins = np.linspace(-np.pi, np.pi, n_bins + 1)
centers = (bins[:-1] + bins[1:]) / 2
idx = np.digitize(theta, bins) - 1
idx[idx == n_bins] = 0  # wrap edge case

prof = np.full(n_bins, np.nan)
for k in range(n_bins):
    m = (idx == k)
    if np.any(m):
        prof[k] = np.nanmean(delta[m])

dmax = float(np.nanmax(prof)*px)
print(f'max denting : {dmax:.4f} mm')

## 8) Visual summary
Overlay the fitted ellipse on the image for a quick visual QA. The title shows eccentricity and the maximum inward dent.

In [ ]:
title = f'Cell Cannister: Eccentricity = {eccentricity:.4f}, Max Dent = {dmax:.4f} mm'
plot.plot_ellipse_on_mask(im, model, title = title)

## 9) Synthetic data example
A small synthetic mask known to have a large dent and elliptical deformation for quick sanity-checking of the pipeline and parameter choices (`px`, thresholds, RANSAC residual).

In [ ]:
# Read image and calcualte a mask
mask = tiff.imread('data/ellipse_dent.tif')
px = 0.0479

# Side-by-side subplots
fig, axes = plt.subplots(1, 1, figsize=(6, 6))

# Iamge Mask Pair: image + overlay
axes.imshow(mask, cmap="gray", interpolation="nearest")
axes.set_title("Ellipse with Dent (Synthetic data)")
axes.axis("off")

plt.tight_layout()
plt.show()

### Helper API
`cn.calculate_can_metrics` runs the same steps end-to-end and returns a structured metrics dictionary (and optionally the fitted ellipse model).

In [ ]:
help(cn.calculate_can_metrics)

### Run the end-to-end helper
Here we request median aggregation for the dent profile, enable RANSAC for robustness, and set `pixel_size` so results print in millimetres.

In [ ]:
# Example
metrics, model = cn.calculate_can_metrics(
    mask,
    use_ransac=True,
    ransac_residual_threshold=1.0,
    ransac_max_trials=3000,
    n_bins=180, 
    profile_agg="median", 
    pixel_size=px,
    return_model=True,
)

print(f"OD = {metrics['diameter']['outer_equiv_units']:.2f} mm")
print(f"ID = {metrics['diameter']['inner_equiv_units']:.2f} mm")
print(f"Thickness mean/min/max = "
      f"{metrics['thickness']['mean_units']:.2f} / "
      f"{metrics['thickness']['min_units']:.2f} / "
      f"{metrics['thickness']['max_units']:.2f} mm")
print(f"Eccentricity = {metrics['ellipse']['eccentricity']:.4f}")
print(f"Max dent = {metrics['dent']['max_units']:.2f} mm")

plot.plot_ellipse_on_mask(mask, model)


The RANSAC model still fits well despite the highly elliptical 'canister' and severe local denting. This allows residuals from the fitted ellipse to be used as a denting metric and shows this to be a robust approach.

### Inspect the returned metrics dictionary
Useful for saving to JSON/CSV or for downstream QA dashboards.

In [ ]:
metrics

# 10) Conclusion
Canister quality is an important indicator of underlying damage in the cell. In this workflow we demonstrated how to extract eccentricity which is an important proxy for deformation (squishing) and denting (i.e. local inwards deformations) that can stress or damage the active materials in the Li-ion cylindrical cell. 